In [1]:
import sys
import os
import numpy as np
import pandas as pd
import sqlalchemy as sa
sys.path.append(os.path.abspath(os.path.join('..','..','..')))
from pudl import pudl, ferc1, eia923, settings, constants, analysis
from pudl import models, models_ferc1, models_eia923
from pudl import clean_eia923, clean_ferc1, clean_pudl
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
pudl_engine  = pudl.db_connect_pudl()

In [3]:
# Convert the generation_eia923 table into a dataframe
g9 = analysis.simple_select('generation_eia923', pudl_engine)
# Get yearly net generation by plant_id, year and generator_id
g9_summed = analysis.yearly_sum_eia(g9 , 'net_generation_mwh')
g9_summed.reset_index(inplace=True)

In [4]:
# Convert the generators_eia860 table into a dataframe
g8 = analysis.simple_select('generators_eia860', pudl_engine)
g8_es = g8[['plant_id','generator_id', 'energy_source_1']]
g8_es = g8_es.rename(columns={'energy_source_1': 'energy_source'})
g8_es.drop_duplicates(['plant_id','generator_id'], inplace=True)

In [5]:
# Merge the generation table with the generator table to include energy_source
net_gen = g9_summed.merge(g8_es, how='left', on=['plant_id','generator_id'])

In [6]:
# Convert the fuel_receipts_costs_eia923 table into a dataframe
frc9 = analysis.simple_select('fuel_receipts_costs_eia923', pudl_engine)
# Get yearly fuel cost by plant_id, year and energy_source
frc9_summed = analysis.yearly_sum_eia(frc9 , 'fuel_cost', columns=['plant_id','report_date','energy_source'])
frc9_summed = frc9_summed.reset_index()

In [7]:
fuel_cost_per_mwh = net_gen.merge(frc9_summed,how='left',on=['plant_id','report_date','energy_source'])
fuel_cost_per_mwh['fuel_cost_per_mwh'] = fuel_cost_per_mwh['fuel_cost'] / fuel_cost_per_mwh['net_generation_mwh']

In [8]:
fuel_cost_per_mwh

,plant_id,report_date,generator_id,net_generation_mwh,energy_source,fuel_cost,fuel_cost_per_mwh
0,3,2009,1,221908.000,BIT,17954.9,0.080911
1,3,2009,2,394031.000,BIT,17954.9,0.045567
2,3,2009,3,1286393.000,BIT,17954.9,0.013958
3,3,2009,4,1626547.000,BIT,17954.9,0.011039
4,3,2009,5,4513101.000,BIT,17954.9,0.003978
5,3,2009,A1ST,1122697.000,NG,10352.0,0.009221
6,3,2009,A2ST,1033733.000,NG,10352.0,0.010014
7,3,2010,1,435334.000,BIT,19198.2,0.044100
8,3,2010,2,538495.000,BIT,19198.2,0.035652
9,3,2010,3,906322.000,BIT,19198.2,0.021183
